In [2]:
#import nflfastpy as nfl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as seabornInstance
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

#Need to update this to latest csv file
csv_file = 'wk20_forecast.csv'
base_data = pd.read_csv(csv_file)

YEARS = [*range(2000, 2023)]

epa_data = pd.DataFrame()

for year in YEARS:  
    i_data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(year) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    epa_data = epa_data.append(i_data, sort=True)

In [3]:
def dynamic_window_ewma(x):
    """
    Calculate rolling exponentially weighted EPA with a dynamic window size
    """
    values = np.zeros(len(x))
    for i, (_, row) in enumerate(x.iterrows()):
        epa = x.epa_shifted[:i+1]
        if row.week > 10:
            values[i] = epa.ewm(min_periods=1, span=row.week).mean().values[-1]
        else:
            values[i] = epa.ewm(min_periods=1, span=10).mean().values[-1]
            
    return pd.Series(values, index=x.index)


rushing_offense_epa = epa_data.loc[epa_data['rush_attempt'] == 1, :]\
.groupby(['posteam', 'season', 'week'], as_index=False)['epa'].mean()

rushing_defense_epa = epa_data.loc[epa_data['rush_attempt'] == 1, :]\
.groupby(['defteam', 'season', 'week'], as_index=False)['epa'].mean()

passing_offense_epa = epa_data.loc[epa_data['pass_attempt'] == 1, :]\
.groupby(['posteam', 'season', 'week'], as_index=False)['epa'].mean()

passing_defense_epa = epa_data.loc[epa_data['pass_attempt'] == 1, :]\
.groupby(['defteam', 'season', 'week'], as_index=False)['epa'].mean()

In [4]:
orig_epa_df_dict = {'rushing_offense_epa':rushing_offense_epa, 'rushing_defense_epa': rushing_defense_epa, 'passing_offense_epa': passing_offense_epa, 'passing_defense_epa': passing_defense_epa}

forecast_epa_df_dict = {}
for key, df in orig_epa_df_dict.items():
    current_sea = df['season'].max()
    forecast_wk = df.loc[(df['season'] == current_sea)]['week'].max() + 1
    team_completed_wk = df.loc[(df['season'] == current_sea)].groupby(df.columns[0])['week'].max()
    df_forecast_name = key+'_forecast'
    forecast_df = pd.DataFrame(columns=df.columns.values)
    if 'offense' in key:
        for idx,ser in df.iterrows():
            team_max = team_completed_wk[ser.posteam]
            if ser.season < current_sea:
                forecast_df = forecast_df.append(ser)
            elif ser.season >= current_sea and ser.week < team_max:
                forecast_df = forecast_df.append(ser)
            elif ser.season >= current_sea and ser.week == team_max:
                forecast_df = forecast_df.append(ser)
                forecast_df = forecast_df.append({'posteam':ser.posteam, 'season':ser.season, 'week':forecast_wk}, ignore_index=True)
        forecast_df['epa_shifted'] = forecast_df.groupby('posteam')['epa'].shift()
        forecast_df['ewma_dynamic_window'] = forecast_df.groupby('posteam').apply(dynamic_window_ewma).values
        forecast_epa_df_dict[df_forecast_name] = forecast_df
    if 'defense' in key:
        for idx,ser in df.iterrows():
            team_max = team_completed_wk[ser.defteam]
            if ser.season < current_sea:
                forecast_df = forecast_df.append(ser)
            elif ser.season >= current_sea and ser.week < team_max:
                forecast_df = forecast_df.append(ser)
            elif ser.season >= current_sea and ser.week == team_max:
                forecast_df = forecast_df.append(ser)
                forecast_df = forecast_df.append({'defteam':ser.defteam, 'season':ser.season, 'week':forecast_wk}, ignore_index=True)
        forecast_df['epa_shifted'] = forecast_df.groupby('defteam')['epa'].shift()
        forecast_df['ewma_dynamic_window'] = forecast_df.groupby('defteam').apply(dynamic_window_ewma).values
        forecast_epa_df_dict[df_forecast_name] = forecast_df

In [5]:
offense_epa = forecast_epa_df_dict['rushing_offense_epa_forecast'].merge(forecast_epa_df_dict['passing_offense_epa_forecast'], on=['posteam', 'season', 'week'], suffixes=('_rushing', '_passing'))\
.rename(columns={'posteam': 'team'})

defense_epa = forecast_epa_df_dict['rushing_defense_epa_forecast'].merge(forecast_epa_df_dict['passing_defense_epa_forecast'], on=['defteam', 'season', 'week'], suffixes=('_rushing', '_passing'))\
.rename(columns={'defteam': 'team'})

epa = offense_epa.merge(defense_epa, on=['team', 'season', 'week'], suffixes=('_offense', '_defense'))

#update team names before joining
epa['team'] = epa['team'].replace('LA','LAR')

epa_basic = epa[['team','season','week','ewma_dynamic_window_rushing_offense','ewma_dynamic_window_passing_offense','ewma_dynamic_window_rushing_defense','ewma_dynamic_window_passing_defense']]

df_home_epa = base_data.merge(epa_basic, how='left', left_on=['team_home_id', 'schedule_season', 'schedule_week'], right_on=['team','season','week'])\
.rename(columns={'ewma_dynamic_window_rushing_offense':'ewma_dynamic_window_rushing_offense_home','ewma_dynamic_window_passing_offense':'ewma_dynamic_window_passing_offense_home',
                 'ewma_dynamic_window_rushing_defense':'ewma_dynamic_window_rushing_defense_home','ewma_dynamic_window_passing_defense':'ewma_dynamic_window_passing_defense_home','team':'home_team_epa'})

df_full_epa = df_home_epa.merge(epa_basic, how='left', left_on=['team_away_id', 'schedule_season', 'schedule_week'], right_on=['team','season','week'])\
.rename(columns={'ewma_dynamic_window_rushing_offense':'ewma_dynamic_window_rushing_offense_away','ewma_dynamic_window_passing_offense':'ewma_dynamic_window_passing_offense_away',
                 'ewma_dynamic_window_rushing_defense':'ewma_dynamic_window_rushing_defense_away','ewma_dynamic_window_passing_defense':'ewma_dynamic_window_passing_defense_away','team':'away_team_epa'})

### Summary Statsitics for Each Target Variables - Home_Team_Win; Home Team Spread; Over_under_line

In [6]:
df_full_epa[['home_team_win','home_team_spread','over_under_line']].describe()

,home_team_win,home_team_spread,over_under_line
count,5617.000000,5621.000000,5621.000000
mean,0.563646,-2.280733,43.633784
std,0.495977,6.039594,4.961218
min,0.000000,-26.500000,30.000000
25%,0.000000,-6.500000,40.500000
50%,1.000000,-3.000000,43.500000
75%,1.000000,3.000000,47.000000
max,1.000000,18.500000,63.500000


In [7]:
#create dummy variables for home and away categorical variables Note: drop first removes ARI from home and away
home_away = df_full_epa[['team_home_id','team_away_id']]

df_cat_dum = pd.get_dummies(data=df_full_epa,columns=['team_home_id','team_away_id'],prefix=['home','away'],drop_first=True)

df = pd.concat([df_cat_dum,home_away],axis=1)

##create Deltas for EPA Data
df['matchup_h_a_delta_rush_off'] = df['ewma_dynamic_window_rushing_offense_home'] - df['ewma_dynamic_window_rushing_offense_away']
df['matchup_h_a_delta_pass_off'] = df['ewma_dynamic_window_passing_offense_home'] - df['ewma_dynamic_window_passing_offense_away']
df['matchup_h_a_delta_rush_def'] = df['ewma_dynamic_window_rushing_defense_home'] - df['ewma_dynamic_window_rushing_defense_away']
df['matchup_h_a_delta_pass_def'] = df['ewma_dynamic_window_passing_defense_home'] - df['ewma_dynamic_window_passing_defense_away']

#Identify the features and targets for each of the three forcasts 

features_categorical = ['home_ATL', 'home_BAL', 'home_BUF',
       'home_CAR', 'home_CHI', 'home_CIN', 'home_CLE', 'home_DAL', 'home_DEN',
       'home_DET', 'home_GB', 'home_HOU', 'home_IND', 'home_JAX', 'home_KC',
       'home_LAC', 'home_LAR', 'home_LV', 'home_MIA', 'home_MIN', 'home_NE',
       'home_NO', 'home_NYG', 'home_NYJ', 'home_PHI', 'home_PIT', 'home_SEA',
       'home_SF', 'home_TB', 'home_TEN', 'home_WAS', 'away_ATL', 'away_BAL',
       'away_BUF', 'away_CAR', 'away_CHI', 'away_CIN', 'away_CLE', 'away_DAL',
       'away_DEN', 'away_DET', 'away_GB', 'away_HOU', 'away_IND', 'away_JAX',
       'away_KC', 'away_LAC', 'away_LAR', 'away_LV', 'away_MIA', 'away_MIN',
       'away_NE', 'away_NO', 'away_NYG', 'away_NYJ', 'away_PHI', 'away_PIT',
       'away_SEA', 'away_SF', 'away_TB', 'away_TEN', 'away_WAS']

#log Classification to predict if home team wins and the probability.
feature_columns_win_loss = ['elo_home_pre', 'elo_away_pre', 'matchup_h_a_delta_elo_pre',
       'home_qb_value_pre', 'away_qb_value_pre', 'matchup_h_a_delta_qb_value_pre',
       'home_qb_adj', 'away_qb_adj', 'matchup_h_a_delta_qb_adj',
       'home_wins','away_wins','matchup_h_a_delta_wins',
       'home_losses','away_losses','matchup_h_a_delta_losses',
       'h_tot_wght_dvoa','a_tot_wght_dvoa','matchup_h_a_delta_tot_wght_dvoa',
       'h_off_wght_dvoa','a_off_wght_dvoa','matchup_h_a_delta_off_wght_dvoa',
       'h_def_wght_dvoa','a_def_wght_dvoa','matchup_h_a_delta_def_wght_dvoa',
       'h_st_wght_dvoa','a_st_wght_dvoa','matchup_h_a_delta_st_wght_dvoa',
       'ewma_dynamic_window_rushing_offense_home','ewma_dynamic_window_rushing_offense_away','matchup_h_a_delta_rush_off',
       'ewma_dynamic_window_passing_offense_home','ewma_dynamic_window_passing_offense_away','matchup_h_a_delta_pass_off',
       'ewma_dynamic_window_rushing_defense_home','ewma_dynamic_window_rushing_defense_away','matchup_h_a_delta_rush_def',
       'ewma_dynamic_window_passing_defense_home','ewma_dynamic_window_passing_defense_away','matchup_h_a_delta_pass_def',
       'schedule_season','schedule_week','home_team_spread']
target_win_loss = 'home_team_win'

#lin regression to predict the home team spread
feature_columns_spread = ['elo_home_pre', 'elo_away_pre', 'matchup_h_a_delta_elo_pre',
       'home_qb_value_pre', 'away_qb_value_pre', 'matchup_h_a_delta_qb_value_pre',
       'home_qb_adj', 'away_qb_adj', 'matchup_h_a_delta_qb_adj',
       'home_wins','away_wins','matchup_h_a_delta_wins',
       'home_losses','away_losses','matchup_h_a_delta_losses',
       'h_tot_wght_dvoa','a_tot_wght_dvoa','matchup_h_a_delta_tot_wght_dvoa',
       'h_off_wght_dvoa','a_off_wght_dvoa','matchup_h_a_delta_off_wght_dvoa',
       'h_def_wght_dvoa','a_def_wght_dvoa','matchup_h_a_delta_def_wght_dvoa',
       'h_st_wght_dvoa','a_st_wght_dvoa','matchup_h_a_delta_st_wght_dvoa',
       'ewma_dynamic_window_rushing_offense_home','ewma_dynamic_window_rushing_offense_away','matchup_h_a_delta_rush_off',
       'ewma_dynamic_window_passing_offense_home','ewma_dynamic_window_passing_offense_away','matchup_h_a_delta_pass_off',
       'ewma_dynamic_window_rushing_defense_home','ewma_dynamic_window_rushing_defense_away','matchup_h_a_delta_rush_def',
       'ewma_dynamic_window_passing_defense_home','ewma_dynamic_window_passing_defense_away','matchup_h_a_delta_pass_def',
       'schedule_season','schedule_week', 'over_under_line']
target_spread = 'home_team_spread'


#lin regression to predict the total score
feature_columns_OU = ['elo_home_pre', 'elo_away_pre', 'matchup_h_a_delta_elo_pre',
       'home_qb_value_pre', 'away_qb_value_pre', 'matchup_h_a_delta_qb_value_pre',
       'home_qb_adj', 'away_qb_adj', 'matchup_h_a_delta_qb_adj',
       'home_wins','away_wins','matchup_h_a_delta_wins',
       'home_losses','away_losses','matchup_h_a_delta_losses',
       'h_tot_wght_dvoa','a_tot_wght_dvoa','matchup_h_a_delta_tot_wght_dvoa',
       'h_off_wght_dvoa','a_off_wght_dvoa','matchup_h_a_delta_off_wght_dvoa',
       'h_def_wght_dvoa','a_def_wght_dvoa','matchup_h_a_delta_def_wght_dvoa',
       'h_st_wght_dvoa','a_st_wght_dvoa','matchup_h_a_delta_st_wght_dvoa',
       'ewma_dynamic_window_rushing_offense_home','ewma_dynamic_window_rushing_offense_away','matchup_h_a_delta_rush_off',
       'ewma_dynamic_window_passing_offense_home','ewma_dynamic_window_passing_offense_away','matchup_h_a_delta_pass_off',
       'ewma_dynamic_window_rushing_defense_home','ewma_dynamic_window_rushing_defense_away','matchup_h_a_delta_rush_def',
       'ewma_dynamic_window_passing_defense_home','ewma_dynamic_window_passing_defense_away','matchup_h_a_delta_pass_def',
       'schedule_season','schedule_week', 'home_team_spread','score_home_ewma','score_away_ewma']
target_OU = 'over_under_line'

#there are 3 NAN rows for EPA data from DF - They need to be dropped
#also dropping score_ewma, because not cleaned in SQL query.

df=df.dropna(subset=['ewma_dynamic_window_passing_defense_home'])



## Forecast 1 - Logistic Regression for HomeTeamWin

#### Update DF name to forecast week and,
#### the season week in the .loc function

In [8]:
htw_features = feature_columns_win_loss + features_categorical

#Make sure train data excludes forecast season
X_train_htw = df.loc[(df['season_week'] != '2022_Division') & (df['home_team_win'].notna()),htw_features] # Features
y_train_htw = df.loc[(df['season_week'] != '2022_Division') & (df['home_team_win'].notna()),target_win_loss] #target

In [9]:
#initialize the model and use cross valication for accuracy, since no train/test spit
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X_train_htw, y_train_htw)

accuracy_scores = cross_val_score(logreg, X_train_htw, y_train_htw, cv=10)

print('Model Accuracy:', np.mean(accuracy_scores))

/Users/tmoazed/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/tmoazed/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

Model Accuracy: 0.6591267500206164


/Users/tmoazed/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Update Cell Below

Update DF name to forecast week and the season week in the .loc function

In [10]:
#Use Model to Make Forcasts and attach them to the Following weeks games
df_2022_wkDivision = df.loc[(df['season_week'] == '2022_Division')].assign(
    predicted_winner = lambda x: logreg.predict(x[htw_features]),
    home_team_win_probability = lambda x: logreg.predict_proba(x[htw_features])[:, 1])

df_2022_wkDivision['predicted_winner'] = df_2022_wkDivision.apply(lambda x: x.team_home_id if x.predicted_winner == 1 else x.team_away_id, axis=1)
df_2022_wkDivision['win_probability'] = df_2022_wkDivision.apply(lambda x: x.home_team_win_probability if x.predicted_winner == x.team_home_id else 1 - x.home_team_win_probability, axis=1)

log_model_columns_basic = ['schedule_date','schedule_season','schedule_week','team_home_id', 'team_away_id','predicted_winner','win_probability','home_team_win_probability']

df_2022_wkDivision[log_model_columns_basic]

,schedule_date,schedule_season,schedule_week,team_home_id,team_away_id,predicted_winner,win_probability,home_team_win_probability
5617,2023-01-21,2022,20,PHI,NYG,PHI,0.740212,0.740212
5618,2023-01-21,2022,20,KC,JAX,KC,0.774940,0.774940
5619,2023-01-22,2022,20,BUF,CIN,BUF,0.635956,0.635956
5620,2023-01-22,2022,20,SF,DAL,SF,0.563783,0.563783


## Forecast 2 - Linear Regression for HomeTeamSpread

In [17]:
hts_features = feature_columns_spread + features_categorical

X_train_hts = df.loc[(df['season_week'] != '2022_Division') & (df['home_team_spread'].notna()) & (df['elo_home_pre'].notna()),hts_features] # Features
y_train_hts = df.loc[(df['season_week'] != '2022_Division') & (df['home_team_spread'].notna()) & (df['elo_home_pre'].notna()),target_spread] #target

lin_reg = LinearRegression()  
lin_model_hts = lin_reg.fit(X_train_hts,y_train_hts)

#define cross-validation method to use
cv = KFold(n_splits=10, random_state=1, shuffle=True)

#use k-fold CV to evaluate model
scores = cross_val_score(lin_reg, X_train_hts, y_train_hts, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

mae = np.mean(np.absolute(scores))

print ("Cross Validated Mean Absolute Errors", mae)

r_squared = cross_val_score(lin_reg, X_train_hts, y_train_hts, scoring='r2',
                         cv=cv, n_jobs=-1)

avg_r_squared = np.mean(r_squared)

print ("Cross Validated R Squared", avg_r_squared)

Cross Validated Mean Absolute Errors 1.8852019623784806
Cross Validated R Squared 0.8339085371393729


In [18]:
df_2022_wkDivision = df_2022_wkDivision.assign(predicted_home_spread = lambda x: lin_model_hts.predict(x[hts_features]))

hts_model_columns_basic = ['schedule_date','schedule_season','schedule_week','team_home_id', 'team_away_id','predicted_winner',
                            'win_probability','home_team_spread','predicted_home_spread']

df_2022_wkDivision[hts_model_columns_basic]

,schedule_date,schedule_season,schedule_week,team_home_id,team_away_id,predicted_winner,win_probability,home_team_spread,predicted_home_spread
5617,2023-01-21,2022,20,PHI,NYG,PHI,0.740212,-7.5,-5.200979
5618,2023-01-21,2022,20,KC,JAX,KC,0.774940,-8.5,-10.160221
5619,2023-01-22,2022,20,BUF,CIN,BUF,0.635956,-5.0,-3.741963
5620,2023-01-22,2022,20,SF,DAL,SF,0.563783,-3.5,-1.122307


## Forecast 3 - Linear Regression for OverUnder

In [19]:
OU_features = feature_columns_OU + features_categorical

X_train_OU = df.loc[(df['season_week'] != '2022_Division') & (df['home_team_spread'].notna()) & (df['score_away_ewma'].notna()) & (df['score_home_ewma'].notna()),OU_features] # Features
y_train_OU = df.loc[(df['season_week'] != '2022_Division') & (df['home_team_spread'].notna()) & (df['score_away_ewma'].notna()) & (df['score_home_ewma'].notna()),target_OU] #target

#lin_reg = LinearRegression()  - Already created above
lin_model_OU = lin_reg.fit(X_train_OU,y_train_OU)

#define cross-validation method to use
cv = KFold(n_splits=10, random_state=1, shuffle=True)

#use k-fold CV to evaluate model
scores_OU = cross_val_score(lin_reg, X_train_OU, y_train_OU, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

mae_OU = np.mean(np.absolute(scores_OU))

print ("Cross Validated Mean Absolute Errors", mae_OU)

r_squared_OU = cross_val_score(lin_reg, X_train_OU, y_train_OU, scoring='r2',
                         cv=cv, n_jobs=-1)

avg_r_squared_OU = np.mean(r_squared_OU)

print ("Cross Validated R Squared", avg_r_squared_OU)

Cross Validated Mean Absolute Errors 1.8600106819046274
Cross Validated R Squared 0.7696919651713593


In [20]:
df_2022_wkDivision= df_2022_wkDivision.assign(predicted_over_under_line = lambda x: lin_model_OU.predict(x[OU_features]))

OU_model_columns_basic = ['schedule_date','schedule_season','schedule_week','team_home_id', 'team_away_id','predicted_winner',
                            'win_probability','home_team_spread','predicted_home_spread','over_under_line','predicted_over_under_line']

df_2022_wkDivision[OU_model_columns_basic]

,schedule_date,schedule_season,schedule_week,team_home_id,team_away_id,predicted_winner,win_probability,home_team_spread,predicted_home_spread,over_under_line,predicted_over_under_line
5617,2023-01-21,2022,20,PHI,NYG,PHI,0.740212,-7.5,-5.200979,48.0,49.625729
5618,2023-01-21,2022,20,KC,JAX,KC,0.774940,-8.5,-10.160221,53.0,48.538548
5619,2023-01-22,2022,20,BUF,CIN,BUF,0.635956,-5.0,-3.741963,48.0,46.378016
5620,2023-01-22,2022,20,SF,DAL,SF,0.563783,-3.5,-1.122307,46.5,46.827149
